# ゼロから作る Deep Learning

## 概要

これは書籍で学んだことをメモするためのノートです。

## 始めに

必要なライブラリをインポートします

- numpy (行列計算)
- matplotlib.pyplot (グラフ描画)
- PIL.Image (画像表示)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image

## 準備運動

- numpyを使用してグラフの描画
- matplotlib.pyplotを使用して画像の描画

In [ ]:
x = np.arange(0, 6, 0.1)

plt.plot(x, np.sin(x), label='sin')
plt.plot(x, np.cos(x), label='cos')
plt.xlabel('x')
plt.xlabel('y')
plt.title('sin & cos')
plt.legend()
plt.show()

plt.imshow(Image.open('img/penguins.jpeg'))

## パーセプトロン

入力値に応じて0か1を返す関数で以下の定義で表される

- `n` : 入力の数

$$
y = \begin{cases}
  0 & x \cdot w + b \lt 0 \\
  1 & x \cdot w + b \ge 0
\end{cases}
$$

出力値をグラフで表すと以下のような線になる

In [ ]:
def step (x) :
    return 0 if x < 0 else 1

def total(x, w, b) :
    return np.dot(x, w) + b

def perseptron(x, w, b) :
    return step(total(x, w, b))

x = np.arange(0, 2, 0.001).reshape(200, 10)
w = np.random.rand(10)
b = -4
plt.ylim(ymin=-0.2, ymax=1.2)
plt.plot([perseptron(x, w, b) for x in x])
plt.show()

## ニューラルネットワーク

パーセプトロンの定義を以下のように書き換える

$$
y = h(x \cdot w + b)
$$

$$
h(a) = \begin{cases}
  0 & a \lt 0 \\
  1 & a \ge 0
\end{cases}
$$

`h(x)`のことを__活性化関数__と呼ぶ

ニューラルネットワークでは活性化関数に以下の関数を使用する

- シグモイド関数
- ReLU

※ パーセプトロンで使用した関数は__ステップ関数__という

### シグモイド関数

$$
h(x) = \frac{1}{1 + \mathrm{e}^{-x}}
$$

### ReLU

$$
h(a) = \begin{cases}
  0 & a \lt 0 \\
  a & a \ge 0
\end{cases}
$$

シグモイド関数、ReLU、ステップ関数を比較すると以下のような線になる

In [ ]:
def sigmoid(x) :
    return 1 / (1 + np.exp(-x))

def relu(x) :
    return np.maximum(0, x)

def nuralnet(x, w, b, func) :
    return func(total(x, w, b))

plt.ylim(ymin=-0.2, ymax=1.2)
plt.plot([nuralnet(x, w, b, sigmoid) for x in x], label='sigmoid')
plt.plot([nuralnet(x, w, b, relu) for x in x], label='ReLU')
plt.plot([perseptron(x, w, b) for x in x], label='perseptron', linestyle='--')
plt.legend()
plt.show()

## 多層ニューラルネットワーク

これまでは[入力]と[出力]だけの2層のネットワークを見てきたが、多層のネットワークも構築可能である

層が増えれば複雑さが増すためより高度な分類が行える

以下に4層ニューラルネットワークの例を示す

In [ ]:
x = np.arange(0, 2, 0.0001).reshape(200, 10, 10)
w = [np.random.rand(10), np.random.rand(10), np.random.rand(1)]
b = [-4, -7, -7]

plt.plot([nuralnet(nuralnet(nuralnet(x, w[0], b[0], sigmoid), w[1], b[1], sigmoid), w[2], b[2], sigmoid) for x in x])
plt.show()

## 出力関数

最後の層で使用する活性関数のことを特別に出力関数という

出力関数には以下の関数がある

- 恒等関数
- シグモイド関数
- ソフトマックス関数

### 恒等関数

常に出力の値が入力値と等しくなる関数

$$
f(x) = x
$$

### ソフトマックス関数

全ての出力層の合計値が1になるため、各ノードの出力値を確立として扱うことができる

$$
y_k = \frac{\mathrm{e}^{a_k}}{\sum^n_{i=1} \mathrm{e}^{a_i}}
$$

In [ ]:
def softmax(x) :
    exp = np.exp(x)
    return exp / np.sum(exp)

x = np.arange(0, 2, 0.0001).reshape(200, 10, 10)
w = [np.random.rand(10), np.random.rand(10, 4), np.random.rand(4, 5)]
b = [-4, -7, -1]

plt.plot([nuralnet(nuralnet(nuralnet(x, w[0], b[0], sigmoid), w[1], b[1], sigmoid), w[2], b[2], softmax) for x in x])
plt.title('Softmax')
plt.show()